# ELP EU

#### Imports

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

from nv_forecasting.load_data.elp_eu_orders_daily import ELPEUOrdersDaily

from nv_forecasting.style import set_style

from nv_forecasting.feature_engineering.add_time_features import add_time_features
from nv_forecasting.feature_engineering.add_lags import add_lags

from nv_forecasting.plots import plot_train_val, plot_train_val_test, plot_prediction

from nv_forecasting.metrics import get_scores, add_scores_to_dict

#### Settings

In [ ]:
TARGET = 'y' # Target column name
DATETIME_COLUMN_NAME = 'ds'
COLUMNS_TO_LAG = [TARGET, 'count_of_orders']
COLUMNS_TO_DROP = [TARGET, 'count_of_orders']

# Feature engineering
LAGS = [182]

# Model training
N_OUTER_SPLITS = 4
N_INNER_SPLITS = 3
N_FINAL_SPLITS = 5
TEST_SIZE = 182
SCORING = 'neg_mean_squared_error'

# Style
set_style()

#### Loading the data

In [ ]:
data_handler = ELPEUOrdersDaily('data\\elp_eu_orders_daily.csv')
df = data_handler.get_dataframe(agg='D')

#### Initial data exploring

In [ ]:
for word in ['day', 'day_of_week', 'month', 'year']:
    fig, ax = plt.subplots(1, 1, figsize=(12, 3))
    sns.lineplot(df, x=getattr(df.index, word), y=df[TARGET])
    plt.xlabel('Date')
    plt.ylabel('$', rotation=0, labelpad=16)
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    plt.title(f'Mean value of sum of orders for each day, aggregated by {word}')
    plt.show()

#### Feature engineering

In [ ]:
add_time_features(df)
add_lags(df, lags=LAGS, columns=COLUMNS_TO_LAG)
df.dropna(inplace=True)

#### Data split for hyperparameter tuning

In [ ]:
X = df.drop(columns=COLUMNS_TO_DROP)
y = df[[TARGET]]

outer_cv = TimeSeriesSplit(n_splits=N_OUTER_SPLITS, test_size=TEST_SIZE)
inner_cv = TimeSeriesSplit(n_splits=N_INNER_SPLITS, test_size=TEST_SIZE)

In [ ]:
fig, ax = plt.subplots(N_OUTER_SPLITS*N_INNER_SPLITS, 1, figsize=(20, 3*N_OUTER_SPLITS*N_INNER_SPLITS), sharex=True)

for outer_fold, (train_and_val_idx, test_idx) in enumerate(outer_cv.split(y)):
    train_and_val, test = y.iloc[train_and_val_idx], y.iloc[test_idx]
    for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(train_and_val)):
        train, val = y.iloc[train_idx], y.iloc[val_idx]
        ax_idx = outer_fold * N_INNER_SPLITS + inner_fold
        plot_train_val_test(ax[ax_idx], TARGET, train, val, test, linewidth=0.5)

#### Hyperparameter tuning (time series nested cross-validation)

In [ ]:
metrics = {'rmse': [], 'mae': [], 'r2': []}
metrics_monthly = {'rmse': [], 'mae': [], 'mape': [], 'r2': []}

param_grid = {
    'n_estimators': [100, 200, 500, 1000, 2000],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 3, 4, 5],
}

fig, ax = plt.subplots(N_OUTER_SPLITS, 1, figsize=(20, 4*N_OUTER_SPLITS), sharex=True)
fig, axm = plt.subplots(N_OUTER_SPLITS, 1, figsize=(20, 4*N_OUTER_SPLITS), sharex=True)

for outer_fold, (train_idx, test_idx) in enumerate(outer_cv.split(y)):
    print(f'--- Outer fold {outer_fold+1} ---')

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    grid_search = GridSearchCV(
        estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
        param_grid=param_grid,
        cv=inner_cv,
        scoring=SCORING,
        n_jobs=-1,
        verbose=3,
        error_score='raise'
    )

    grid_search.fit(X_train, y_train[TARGET])

    print('Best parameters:')
    for key, value in grid_search.best_params_.items():
        print(f'{key}: {value}')
    print()

    y_pred = pd.DataFrame(grid_search.predict(X_test),
                          columns=['y'],
                          index=y_test.index)
    
    scores = get_scores(y_test, y_pred)
    add_scores_to_dict(metrics, scores)

    plot_prediction(ax[outer_fold], TARGET, y_train, y_test, y_pred, linewidth=0.5)

    y_train = y_train.resample('ME').mean()
    y_test = y_test.resample('ME').mean()
    y_pred = y_pred.resample('ME').mean()

    scores = get_scores(y_test, y_pred)
    add_scores_to_dict(metrics_monthly, scores)

    plot_prediction(axm[outer_fold], TARGET, y_train, y_test, y_pred)

In [ ]:
pd.DataFrame(metrics).mean()

In [ ]:
pd.DataFrame(metrics_monthly).mean()

#### Data split for final training

In [ ]:
final_cv = TimeSeriesSplit(N_FINAL_SPLITS, test_size=TEST_SIZE)

fig, ax = plt.subplots(N_FINAL_SPLITS, 1, figsize=(20, 3*N_FINAL_SPLITS), sharex=True)

for fold, (train_idx, val_idx) in enumerate(final_cv.split(y)):
    train, val = y.iloc[train_idx], y.iloc[val_idx]

    plot_train_val(ax[fold], TARGET, train, val, linewidth=0.5)

#### Final model training

In [ ]:
final_grid = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    cv=final_cv,
    scoring=SCORING,
    n_jobs=-1,
    verbose=3,
    error_score='raise'
)

final_grid.fit(X, y[TARGET])

print('Best parameters:')
for key, value in final_grid.best_params_.items():
    print(f'{key}: {value}')

prod_model = final_grid.best_estimator_

#### Feature importance analysis

In [ ]:
fi = pd.DataFrame(prod_model.feature_importances_,
                  index=prod_model.feature_names_in_,
                  columns=['val']).sort_values(by='val')

fi.plot(kind='barh', figsize=(8, 4))